In [3]:
import requests
import json
import pandas as pd
import time
import numpy as np

In [4]:

fileName = "events.csv"
region_df = pd.read_csv(fileName)
#region_df=region_df.replace(np.nan,"na")
#region_df = region_df[region_df['price']!='na']
#region_df['country']=region_df['country'].replace(',','na')
region_df = region_df.reset_index()
region_df = region_df.rename(columns={'index':'event_id'})
#region_df = region_df.fillna(0)
region_df

,event_id,state,city,event_name,group_name,lat,lng,venue_event_link
0,0,NJ,Trenton,THURSDAY AFTERNOON AEROBIC HIKE,Washington Crossing Park Association of NJ,40.2236,-74.7641,https://www.meetup.com/washingtoncrossingstate...
1,1,NJ,Trenton,Fly-In at Ellisdale Fields,NJ Drone User Group,40.2236,-74.7641,https://www.meetup.com/NJ-Dug/events/275376752
2,2,NJ,Trenton,SMART RECOVERY at LifeBack,S.M.A.R.T. Recovery at LifeBack,40.2236,-74.7641,https://www.meetup.com/S-M-A-R-T-Recovery-at-L...
3,3,NJ,Trenton,Lifeback's Helping Families Affected By Addiction,Lifeback's Helping Families Affected By Addiction,40.2236,-74.7641,https://www.meetup.com/Lifebacks-Helping-Famil...
4,4,NJ,Trenton,Flying Solo Support Group,Flying Solo Support Group,40.2236,-74.7641,https://www.meetup.com/meetup-group-uLdxSObt/e...
...,...,...,...,...,...,...,...,...
1377,1377,PA,Levittown,SMART RECOVERY at LifeBack,S.M.A.R.T. Recovery at LifeBack,40.1537,-74.8530,https://www.meetup.com/S-M-A-R-T-Recovery-at-L...
1378,1378,PA,Levittown,Lifeback's Helping Families Affected By Addiction,Lifeback's Helping Families Affected By Addiction,40.1537,-74.8530,https://www.meetup.com/Lifebacks-Helping-Famil...
1379,1379,PA,Levittown,Let's meet up and sing!,BucksMont Squires of Song,40.1537,-74.8530,https://www.meetup.com/BucksMont-Squires-of-So...
1380,1380,PA,Bloomsburg,Open Ukulele Jam,Danville Ukulele Meetup,41.0027,-76.4561,https://www.meetup.com/Danville-Ukulele-Meetup...


In [13]:
# Create a region id table based on (city,state,lat and lng) and save to region.csv to be loaded in postgresql

region_table1 = region_df.groupby(['city','state','lat','lng']).size().reset_index().rename(columns={0:'count'})
region_table2 = region_table1.reset_index()
#region_table3 = region_table2(columns={"index": "event_id"})
region_table2=region_table2.drop(['count'] , axis='columns')
region_table2

,index,city,state,lat,lng
0,0,Albany,NY,42.6664,-73.7987
1,1,Allentown,PA,40.5961,-75.4756
2,2,Altoona,PA,40.5082,-78.4007
3,3,Atlantic City,NJ,39.3797,-74.4527
4,4,Bayonne,NJ,40.6661,-74.1141
...,...,...,...,...,...
57,57,White Plains,NY,41.0220,-73.7549
58,58,Wilkes Barre,PA,41.2468,-75.8759
59,59,Williamsport,PA,41.2398,-77.0371
60,60,Yonkers,NY,40.9466,-73.8674


In [14]:
#3##save teh region table for postgres
region_table2.to_csv('region_table.csv', encoding='utf-8', index=False)

In [ ]:
new_df.sort_values(by='Region_Id')

In [74]:
# Create dataframe for merchants to be saved as a csv (merchant.csv) to be loaded in postresql
merchant_address = region_df.groupby(['merchant','address']).nunique()
merchant_address = merchant_address.drop(['wine_id','name','country', 'state', 'region', 'varietal','price', 'description'], axis='columns')
merch_add_df=merchant_address.reset_index()
merch_add_df

,merchant,address
0,Chatham BottleKing,"41 Watchung Ave, Chatham, NJ, 07928"
1,Gary's Wayne,"1308 NJ-23, Wayne, NJ 07470, USA"
2,Petrocks Liquors,"419 Amwell Road Hillsborough, NJ 08844"


In [76]:
merch_add_df.to_csv('merchant.csv', encoding='utf-8', index=False)

In [68]:
# Create a merchant_wine dataframe to be saved as a csv (merchant_wine.csv) to be loaded in postgresql
merchant_wine_df=region_df[['merchant','wine_id']]
merchant_wine_df.reset_index(drop=True, inplace=True)
merchant_wine_df

,merchant,wine_id
0,Gary's Wayne,0
1,Gary's Wayne,1
2,Gary's Wayne,2
3,Gary's Wayne,3
4,Gary's Wayne,4
...,...,...
6081,Petrocks Liquors,6088
6082,Petrocks Liquors,6089
6083,Petrocks Liquors,6090
6084,Petrocks Liquors,6091


In [69]:
merchant_wine_df.to_csv("merchant_wine.csv",encoding='utf-8', index=False)

In [73]:
# Create and clean a dataframe which will be saved as a csv (wine.csv) to be loaded in postgresql 

event_df = pd.merge(region_df, region_table2,  how='left', left_on=['city','state','lat','lng'], right_on = ['country','region','state'])
new_df = new_df.drop(['country','region', 'state','merchant', 'address' ] , axis='columns')
new_df['region_Id'] = new_df['region_Id'].astype(int)
new_df


,wine_id,name,varietal,price,description,region_Id
0,0,Angels Landing Cabernet Sauvignon,Cabernet Sauvignon,22.99,"This Cabernet has it all. Ripe dark fruit, str...",744
1,1,Angels Landing Merlot Napa,Merlot,16.99,"A rich mix of black fruit, hints of dark choco...",744
2,2,Antale Veneto Rosso,Corvina,11.99,Producing every style of wine and with great s...,481
3,3,Articulate Cabernet Sauvignon Walla Walla,Cabernet Sauvignon,14.99,na,777
4,4,Baus Family Cabernet Sauvignon Private Reserve,Cabernet Sauvignon,16.99,"A full-bodied and rich Cabernet, this wine is ...",711
...,...,...,...,...,...,...
6081,6088,Candoni Pinot Grigio Organic,Pinot Gris,11.99,Our Organic Pinot Grigio is made from 100% Org...,432
6082,6089,Canei Peach,na,7.99,Hand it over to an Italian to make an exciting...,522
6083,6090,Kendall Jackson Cabernet Grand Reserve,Cabernet Sauvignon,21.99,Grand Reserve is a winemakerâs blend of our ...,754
6084,6091,Bacchus Cabernet Sauvignon,Cabernet Sauvignon,9.99,David Gordon has been the wine director at Tri...,782
